# Testing cocoa.geo.GeoManager

## Importing data
If executed locally

In [1]:
import sys
sys.path.insert(1, '..')
from cocoa.geo import GeoManager as gm

If executed on a server (e.g. [Google Colab](https://colab.research.google.com/) ), you should install cocoa via `pip`

In [ ]:
!pip install #... to complete…

## Getting some help

In [2]:
help(gm)

Help on class GeoManager in module cocoa.geo:

class GeoManager(builtins.object)
 |  GeoManager(standard='iso2')
 |  
 |  GeoManager class definition. No inheritance from any other class.
 |  
 |  It should raise only CocoaError and derived exceptions in case
 |  of errors (see cocoa.error)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, standard='iso2')
 |      __init__ member function, with default definition of
 |      the used standard. To get the current default standard,
 |      see get_list_standard()[0].
 |  
 |  first_db_translation(self, w, db)
 |      This function helps to translate from country name to
 |      standard for specific databases. It's the first step
 |      before final translation.
 |      
 |      One can easily add some database support adding some new rules
 |      for specific databases
 |  
 |  get_list_db(self)
 |      return supported list of database name for translation of
 |      country names to standard.
 |  
 |  get_list_output(self)
 |   

## Usage

In [4]:
g=gm() # Create the instance of GeoManager 

### Suppported options

In [8]:
g.get_list_db()

[None, 'jhu', 'worldometers', 'owid']

In [9]:
g.get_list_output()

['list', 'dict', 'pandas']

In [10]:
g.get_list_standard()

['iso2', 'iso3', 'name', 'num']

In [11]:
g.get_standard()

'iso2'

### Changing the default output standard

In [12]:
g.set_standard('name')

'name'

In [13]:
g2=gm('iso3')
g2.get_standard()

'iso3'

In [15]:
g.get_standard()

'name'

### Converting countries to standardized names

In [18]:
g.to_standard('england') # single country

['United Kingdom']

In [19]:
g.to_standard(['esp','it']) # list of countries

['Spain', 'Italy']

In [24]:
from cocoa.error import * 
try:
    z=g.to_standard('European Union') # the name does not exist as a country
except CocoaError:
    print('The input name is unknown, try to interpret as a region')
    z=g.to_standard('European Union',interpret_region=True)
print(z)

The input name is unknown, try to interpret as a region
['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Germany', 'Denmark', 'Spain', 'Estonia', 'Finland', 'France', 'Greece', 'Croatia', 'Hungary', 'Ireland', 'Italy', 'Lithuania', 'Luxembourg', 'Latvia', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Sweden']


In [37]:
g.to_standard(['usa','South America','russia'],interpret_region=True) # mix between region and countries

['United States',
 'Argentina',
 'Bolivia, Plurinational State of',
 'Brazil',
 'Bouvet Island',
 'Chile',
 'Colombia',
 'Ecuador',
 'Falkland Islands (Malvinas)',
 'French Guiana',
 'Guyana',
 'Peru',
 'Paraguay',
 'South Georgia and the South Sandwich Islands',
 'Suriname',
 'Uruguay',
 'Venezuela, Bolivarian Republic of',
 'Russian Federation']

In [25]:
g.to_standard('french') # difficult interpretation, getting first item, a warning appears

../cocoa/geo.py:163: UserWarning: Caution. More than one country match the key "french" : ['France, ', 'French Guiana, ', 'French Polynesia, ', 'French Southern Territories, ', 'Saint Martin (French part), '], using first one.

  warnings.warn('Caution. More than one country match the key "'+\


['France']

### Conversion for specific databases

In [33]:
try:
    z=g.to_standard('Congo (Kinshasa)')
except CocoaError:
    print('Try to interpret within the JHU database country name usage')
    z=g.to_standard('Congo (Kinshasa)',db='jhu')
z

Try to interpret within the JHU database country name usage


['Congo, The Democratic Republic of the']

### Other outputs

In [79]:
g.to_standard(['fr','spain','england'],output='dict')

{'fr': 'France', 'spain': 'Spain', 'england': 'United Kingdom'}

In [80]:
g.to_standard(['fr','spain','england'],output='pandas')

,inputname,name
0,fr,France
1,spain,Spain
2,england,United Kingdom


## Management of errors

As far as possible, errors are managed within the `cocoa.error` framework. `CocoaError` should be raised.

In [16]:
g.set_standard('oups') # the standard is not in the standard list 

CocoaKeyError: 'GeoManager.set_standard error, "oups not managed. Please see get_list_standard() function'

In [17]:
g3=gm('hi!') # idem

CocoaKeyError: 'GeoManager.set_standard error, "hi! not managed. Please see get_list_standard() function'

In [43]:
g.to_standard('Congo (Kinshasa)',db='another base') # unknown base

CocoaDbError: Unknown database "another base" for translation to standardized location names. See get_list_db() or help.

In [49]:
g.to_standard('aus',output='another output') # bad output type

CocoaKeyError: 'Incorrect output type. See get_list_output() or help.'

In [50]:
g.to_standard('Europe',interpret_region=1) # bad type (boolean required) for interpret_region option

CocoaTypeError: The interpret_region argument is a boolean, not a <class 'int'>

In [52]:
g.to_standard('elsewhere') # unknown country

CocoaLookupError: No country match the key "elsewhere". Error.

In [74]:
g.to_standard('European Union',output='dict',interpret_region=True) # cannont create dict or pandas output with interpret_region=True

CocoaKeyError: 'The interpret_region True argument is incompatible with non list output option.'